In [6]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('train.csv')
df['question2'] = df['question2'].astype(str)
df['question1'] = df['question1'].astype(str)

In [38]:


def stemming(row):
    stops = set(stopwords.words("english"))
    tokens = word_tokenize(row)
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in tokens if word not in stops]
    return stemmed

def tfidf_tokenizer(row):
    tokens = word_tokenize(row)
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in tokens]
    return stemmed

In [34]:
df['q1_stemmed'] = df.question1.apply(stemming)
df['q2_stemmed'] = df.question2.apply(stemming)

In [45]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_stemmed,q2_stemmed,tfidf_dist
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[what, step, step, guid, invest, share, market...","[what, step, step, guid, invest, share, market...",0.094450
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[what, stori, kohinoor, (, koh-i-noor, ), diam...","[what, would, happen, indian, govern, stole, k...",0.430166
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[how, I, increas, speed, internet, connect, us...","[how, internet, speed, increas, hack, dn, ?]",0.549732
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[whi, I, mental, lone, ?, how, I, solv, ?]","[find, remaind, [, math, ], 23^, {, 24, }, [, ...",0.908700
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[which, one, dissolv, water, quikli, sugar, ,,...","[which, fish, would, surviv, salt, water, ?]",0.771173


In [44]:
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidf_dis(row):
    try:
        vectorizer = TfidfVectorizer(tokenizer=tfidf_tokenizer, stop_words='english')
        vec = vectorizer.fit_transform([row['question1'], row['question2']])
        vec = vec.todense()
        return cosine(vec[0], vec[1])
    except:
        return 1.0
    
# def tfidf_dis(row):
#     vectorizer = TfidfVectorizer(tokenizer=tfidf_tokenizer, stop_words='english')
#     vec = vectorizer.fit_transform([row['question1'], row['question2']])
#     vec = vec.todense()
#     return cosine(vec[0], vec[1])

df['tfidf_dist'] = df.apply(tfidf_dis, axis=1)

/home/leonardoli525/anaconda3/lib/python3.5/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [46]:
df.to_csv('clean.csv')